# Practica 3

Esta tercera practica consiste en obtener información simple y descriptiva que de indicios sobre los patrones que tiene nuestro problema. La idea es responder preguntas simples que ayuden posteriormente a formular nuestras hipótesis para futuras prácticas.

En la practica anterior se estructuro los datos de acuerdo a las distintas combinaciones de parámetros del diseño de experimentos para cada método de solución, sin embargo, para facilitarnos la tarea se trabaja con los datos crudos.